In [ ]:
import pathlib
import polars as pl

datadir = pathlib.Path.cwd().parent / "data"

pl_hotel       = pl.read_parquet(datadir / "hotel.parquet")
pl_customer    = pl.read_parquet(datadir / "customer.parquet")
pl_reservation = pl.read_parquet(datadir / "reservation.parquet")

# 6章 集約
## 6-1 データ全体の集約と代表的な集約関数
### Q：予約履歴の各種集計値の算出


#### Awesome

In [ ]:
(
    pl_reservation
    # （1）キャンセル済みではない予約の抽出
    .filter(pl.col("status") != "canceled")
    # （2）集計値の計算
    .select([
        pl.col("reservation_id").len().alias("reservation_cnt"),   # カウント
        pl.col("total_price").sum().alias("sales"),                # 総和
        pl.col("total_price").mean().alias("mean_sales"),          # 平均値
        pl.col("total_price").min().alias("min_sales"),            # 最小値
        pl.col("total_price").max().alias("max_sales"),            # 最大値
        pl.col("total_price").var().alias("var_sales"),            # 不偏分散
        pl.col("total_price").std().alias("std_sales"),            # 不偏標準偏差
    ])
)

### Q：予約顧客のユニークカウントの算出


#### Awesome

In [ ]:
(
    pl_reservation
    # （1）キャンセル済みではない予約の抽出
    .filter(pl.col("status") != "canceled")
    # （2）ユニークカウントの計算
    .select(pl.col("customer_id").n_unique())
)

### Q：予約単価の中央値およびパーセンタイル値の算出


#### Awesome

In [ ]:
(
    pl_reservation
    # （1）キャンセル済みではない予約の抽出
    .filter(pl.col("status") != "canceled")
    # （2）中央値、パーセンタイル値の計算
    .select([
        pl.col("total_price").median().alias("median_sales"),
        pl.col("total_price").quantile(0.25, interpolation="linear")
            .alias("p25_sales"),
        pl.col("total_price").quantile(0.75, interpolation="linear")
            .alias("p75_sales"),
    ])
)

### Q：ホテルごとの宿泊人数の最頻値の算出


#### Awesome

In [ ]:
(
    pl_reservation
    # （1）キャンセル済みではない予約の抽出
    .filter(pl.col("status") != "canceled")
    # （2）グループごとに最頻値を取得
    .group_by("hotel_id").agg(pl.col("people_num").mode().first().alias("mode_people_num"))
)

## 6-2 グループごとの集約
### Q：ホテルごとの売上の集計


#### Awesome

In [ ]:
(
    pl_reservation
    # （1）キャンセル済みではない予約の抽出
    .filter(pl.col("status") != "canceled")
    # （2）hotel_idの値ごとにtotal_priceの総和を計算
    .group_by("hotel_id").agg(pl.col("total_price").sum())
)

### Q：ホテルごと・顧客ごとの予約数の集計


#### Awesome

In [ ]:
(
    pl_reservation
    # （1）キャンセル済みではない予約の抽出
    .filter(pl.col("status") != "canceled")
    # （2）hotel_idとcustomer_idの値の組ごとにデータ数をカウント
    .group_by(["hotel_id", "customer_id"]).agg(pl.len())
)

## 6-3 数値の区間ごとの集約
### Q：等間隔の価格帯ごとにホテル数を集計


#### Awesome

In [ ]:
(
    pl_hotel
    # （1）unit_priceを数値区間に丸めた列を作成し、それを集約キーとしてgroup by
    .group_by((pl.col("unit_price") / 5000).floor().cast(pl.Int32) * 5000)
    # （2）グループごとにデータ数をカウント
    .agg(pl.len())
)

### Q：非等間隔の価格帯ごとにホテル数を集計


#### Awesome

In [ ]:
(
    pl_hotel
    # （1）unit_priceを丸めた数値区間データを作成し、それ集約キーとしてgroup by
    .group_by(
        pl.when(pl.col("unit_price") < 5000).then(0)
        .when(pl.col("unit_price") < 10000).then(5000)
        .when(pl.col("unit_price") < 20000).then(10000)
        .when(pl.col("unit_price") < 30000).then(20000)
        .otherwise(30000)
        .alias("unit_price")
    )
    # （2）数値区間ごとにデータ数をカウント
    .agg(pl.len())
)

## 6-4 時間の区間ごとの集約
### Q：月ごとの売上の集計


#### Awesome

In [ ]:
(
    pl_reservation
    # （1）キャンセル済みではない予約の抽出
    .filter(pl.col("status") != "canceled")
    # （2）checkout_dateの月部分を切り出したデータを用いてgroup by
    .group_by(pl.col("checkout_date").dt.truncate("1mo"))
    # （3）月ごとにtotal_priceの総和を計算
    .agg(pl.col("total_price").sum())
)

## 6-5 条件を満たす行の存在判定
### Q：チェックインの7日以内に予約をキャンセルしたことがある顧客の判定


#### Awesome

In [ ]:
(
    pl_reservation
    #（1）customer_idを集約キーとしてgroup by
    .group_by("customer_id")
    #（2）customer_idの値ごとに、キャンセル済み、かつcanceled_atがcheckin_dateの7日以内、
    #    というデータが1件以上存在するかどうかを集計
    .agg(
        (
            (pl.col("status") == "canceled")
            & ((pl.col("checkin_date") - pl.col("canceled_at")).dt.total_days() <= 7)
        ).max()
    )
)

## 6-6 条件を満たす行のみの集約
### Q：顧客ごとの売上とキャンセル率の算出


#### Awesome

In [ ]:
(
    pl_reservation
    .group_by("customer_id").agg([
        #（1）customer_idの値ごとに、未キャンセル予約を抽出してtotal_priceの総和を計算
        pl.col("total_price").filter(pl.col("status") != "canceled").sum(),
        #（2）customer_idの値ごとに、キャンセル済み予約のカウントと全データ数の除算を計算
        (pl.col("reservation_id").filter(pl.col("status") == "canceled").len()
            / pl.len()).alias("cancel_rate"),
    ])
)